In [2]:
import numpy as np
import open3d as o3d
import copy
import math

from collections import defaultdict
from scipy.spatial import Delaunay
from tqdm import tqdm

In [3]:
class HalfEdge:
    def __init__(self):
        self.origin = None  # Начальная вершина
        self.twin = None    # Парное полурёбро
        self.face = None    # Принадлежащая грань
        self.next = None    # Следующее полурёбро в грани

    def __repr__(self):
        return f"HalfEdge({self.origin}, {self.twin.origin})"
        

class Face:
    def __init__(self):
        self.edge = None  # Любое граничное полурёбро

    def vertex_indices(self):
        if self.edge is None:
            return
        
        current = self.edge
        while True:
            yield current.origin
            current = current.next
            
            if current is self.edge:
                break

class DCEL:
    def __init__(self, vertices, edges, faces):
        self.vertices = vertices
        self.edges = edges
        self.faces = faces

In [4]:
# Хеш-функция для вершин (учет возможных погрешностей float)
def vertex_hash(v):
    return tuple(np.round(v, 6))

def segment_hash(seg):
    return tuple(sorted(seg))


def repair_mesh(mesh):
    # Проверяем, есть ли проблема с вершинами
    vertices = np.asarray(mesh.vertices)
    triangles = np.asarray(mesh.triangles)

    # Создаем словарь для устранения дубликатов вершин
    vertex_map = {}
    new_vertices = []
    new_triangles = []

    # Строим отображение старых вершин в новые
    for idx, v in enumerate(vertices):
        v_hash = vertex_hash(v)
        if v_hash not in vertex_map:
            vertex_map[v_hash] = len(new_vertices)
            new_vertices.append(v)

    # Перестраиваем треугольники с новыми индексами
    for tri in triangles:
        new_tri = [
            vertex_map[vertex_hash(vertices[tri[0]])],
            vertex_map[vertex_hash(vertices[tri[1]])],
            vertex_map[vertex_hash(vertices[tri[2]])]
        ]
        new_triangles.append(new_tri)

    # Создаем новую сетку
    repaired_mesh = o3d.geometry.TriangleMesh()
    repaired_mesh.vertices = o3d.utility.Vector3dVector(np.array(new_vertices))
    repaired_mesh.triangles = o3d.utility.Vector3iVector(
        np.array(new_triangles)
    )

    # Вычисляем нормали для корректного отображения
    repaired_mesh.compute_vertex_normals()

    return repaired_mesh


def load_and_repair_mesh(filepath):
    mesh = o3d.io.read_triangle_mesh(filepath)
    repaired_mesh = repair_mesh(mesh)

    return repaired_mesh

In [5]:
mesh_a = load_and_repair_mesh("model.stl")
mesh_b = load_and_repair_mesh("sphere.stl")

if not mesh_a.has_vertex_normals():
    mesh_a.compute_vertex_normals()
if not mesh_b.has_vertex_normals():
    mesh_b.compute_vertex_normals()

In [6]:
def triangle_area(v1, v2, v3):
    """Вычисление площади треугольника"""
    return 0.5 * np.linalg.norm(np.cross(v2 - v1, v3 - v1))

def center_of_mass(mesh):
    """Вычисление центра масс меша"""
    triangles = np.asarray(mesh.triangles)
    vertices = np.asarray(mesh.vertices)
    
    # Получаем вершины каждого треугольника
    tri_vertices = vertices[triangles]
    
    # Центры треугольников
    centers = np.mean(tri_vertices, axis=1)
    
    # Площади треугольников
    areas = np.array([triangle_area(t[0], t[1], t[2]) for t in tri_vertices])
    
    # Центр масс
    return np.sum(centers * areas.reshape(-1, 1), axis=0) / np.sum(areas)

In [7]:
ITERS = 10

def normalize(vector):
    if not np.any(vector):
        raise ZeroDivisionError

    return vector / np.linalg.norm(vector)


def get_adjacent_vertices(mesh):
    """Построение словаря смежных вершин"""
    triangles = np.asarray(mesh.triangles)
    adjacency_dict = defaultdict(set)

    for tri in triangles:
        v0, v1, v2 = tri

        # Добавляем связи между вершинами треугольника
        adjacency_dict[v0].update([v1, v2])
        adjacency_dict[v1].update([v0, v2])
        adjacency_dict[v2].update([v0, v1])

    return adjacency_dict


def relax_mesh(mesh, iter_limit=ITERS):
    """Релаксация меша с сохранением топологии"""
    # Получаем копию исходного меша
    relaxed_mesh = copy.deepcopy(mesh)
    
    # Получаем вершины и треугольники
    vertices = np.asarray(relaxed_mesh.vertices)
    adj = get_adjacent_vertices(mesh)
    
    # Релаксация
    for _ in range(iter_limit):
        prev_vertices = copy.deepcopy(vertices)

        for i in range(len(vertices)):
            neighbors = list(adj[i])
            try:
                vertices[i] = normalize(np.mean(prev_vertices[neighbors], axis=0))
            except:
                pass
    
    # Обновляем нормали
    relaxed_mesh.compute_vertex_normals()
    return relaxed_mesh


def parametrize_mesh(mesh):
    """Параметризация меша на сфере"""
    # Создаем копию меша
    mesh_copy = copy.deepcopy(mesh)

    # Вычисляем центр масс
    center = center_of_mass(mesh_copy)

    # Переносим вершины в начало координат и нормализуем
    vertices = np.asarray(mesh_copy.vertices)
    vertices -= center
    norms = np.linalg.norm(vertices, axis=1)
    norms[norms == 0] = 1  # Избегаем деления на ноль #TODO: troubleshoot
    vertices = vertices / norms.reshape(-1, 1)

    # Обновляем вершины и нормали
    mesh_copy.vertices = o3d.utility.Vector3dVector(vertices)
    mesh_copy.compute_vertex_normals()

    # Применяем релаксацию
    return relax_mesh(mesh_copy, ITERS)

In [8]:
def overlap_meshes(source_mesh, target_mesh):
    source_pcd = o3d.geometry.PointCloud()
    source_pcd.points = source_mesh.vertices
    target_pcd = o3d.geometry.PointCloud()
    target_pcd.points = target_mesh.vertices
    
    source_pcd.estimate_normals()
    target_pcd.estimate_normals()

    trans_init = np.identity(4)
    threshold = 0.02 * 2
    reg_p2l = o3d.pipelines.registration.registration_icp(
        source_pcd, target_pcd, threshold, trans_init,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=200)
    )
    source_pcd.transform(reg_p2l.transformation)
    source_mesh.vertices = source_pcd.points

In [9]:
def add_wireframe_to_geo(mesh, geo, color = [0, 0, 0]):
    wireframe = o3d.geometry.LineSet.create_from_triangle_mesh(mesh)
    pcd = o3d.geometry.PointCloud()
    pcd.points = mesh.vertices

    pcd.paint_uniform_color(color)
    wireframe.paint_uniform_color(color)

    geo.append(wireframe)
    geo.append(pcd)


def add_dcell_to_geo(dcel, geo, color = [0, 0, 0]):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(dcel.vertices)
    pcd.paint_uniform_color(color)

    lineset = o3d.geometry.LineSet()
    lineset.points = pcd.points
    lineset.lines = o3d.utility.Vector2iVector(np.array([[e.origin, e.twin.origin] for e in dcel.edges]))
    lineset.paint_uniform_color(color)

    geo.append(lineset)
    geo.append(pcd)

In [10]:
def segments(mesh):
    seen = set()

    for tri in np.asarray(mesh.triangles):
        segments = (
            tuple(sorted([tri[0], tri[1]])),
            tuple(sorted([tri[1], tri[2]])),
            tuple(sorted([tri[2], tri[0]]))
        )
        for seg in segments:
            if seg not in seen:
                seen.add(seg)
                yield seg


def arcs(mesh):
    vertices = np.asarray(mesh.vertices)


    for seg in segments(mesh):
        yield (vertices[seg[0]], vertices[seg[1]])


def find_arcs_intersection(arc_a, arc_b):
    normal_a = np.cross(arc_a[0], arc_a[1])
    if np.dot(arc_a[0], arc_b[0]) < 0:
        return None

    normal_b = np.cross(arc_b[0], arc_b[1])
    d = np.cross(normal_a, normal_b)

    if np.isclose(np.linalg.norm(d), 0):
        return None

    if np.dot(d, arc_a[0]) < 0:
        d = -d

    if np.dot(np.cross(arc_a[0], d), np.cross(arc_a[1], d)) >= 0:
        return None

    if np.dot(np.cross(arc_b[0], d), np.cross(arc_b[1], d)) >= 0:
        return None

    return normalize(d)


def find_parametrized_mesh_intersections(mesh_a: o3d.geometry.TriangleMesh, mesh_b: o3d.geometry.TriangleMesh):

    result = np.ndarray((0, 3), dtype=float)


    for arc_a in tqdm(arcs(mesh_a), total=len(mesh_a.triangles) * 3):


        for arc_b in arcs(mesh_b):


            inter = find_arcs_intersection(arc_a, arc_b)
            if inter:
                result = np.append(result, [inter])

    return result

In [11]:
def mesh_from_dcel(dcel: DCEL):
    triangles = []
    
    for face in dcel.faces:            
        triangles.append(list(face.vertex_indices()))

    mesh = o3d.geometry.TriangleMesh()
    mesh.vertices = o3d.utility.Vector3dVector(dcel.vertices)
    mesh.triangles = o3d.utility.Vector3iVector(np.array(triangles))
    mesh.compute_vertex_normals()

    return mesh

In [12]:
def tangent_angle(normal, direction):
    """Вычисляет угол направления в касательной плоскости вершины."""
    # Шаг 1: Выбираем опорный вектор, не коллинеарный нормали
    if abs(normal[0]) < 0.1 and abs(normal[1]) < 0.1:
        ref = (1, 0, 0)  # Если нормаль близка к оси Z
    else:
        ref = (0, 0, 1)
    
    # Шаг 2: Строим ортонормированный базис (u, v)
    u = np.cross(normal, ref)
    u = normalize(u)
    
    # Если u нулевой, пробуем другой ref
    if math.sqrt(u[0]**2 + u[1]**2 + u[2]**2) < 1e-6:
        ref = (1, 0, 0) if abs(normal[0]) < 0.5 else (0, 1, 0)
        u = np.cross(normal, ref)
        u = normalize(u)
    
    v = np.cross(normal, u)  # Уже ортонормирован
    
    # Шаг 3: Проекция вектора направления
    w_proj = direction  # Мы не вычитаем нормальную компоненту, т.к. dot(w_proj, v) даст то же
    x = np.dot(w_proj, u)
    y = np.dot(w_proj, v)
    
    # Шаг 4: Вычисляем угол [0, 2π]
    angle = math.atan2(y, x)
    return angle + 2*math.pi if angle < 0 else angle


def build_dcel(vertices, segments):
    vertex_objs = np.array(vertices)
    edges = np.ndarray((0, ), dtype=HalfEdge)
    faces = np.ndarray((0, ), dtype=Face)
    edge_map = defaultdict(list)

    for start, end in segments:
        he1 = HalfEdge()
        he2 = HalfEdge()

        he1.twin = he2
        he2.twin = he1

        he1.origin = start
        he2.origin = end

        edges = np.append(edges, [he1, he2])

        edge_map[start].append(he1)
        edge_map[end].append(he2)

    for key in edge_map:
        edge_map[key].sort(key=lambda he: tangent_angle(vertices[key], vertices[he.twin.origin] - vertices[key]))

    for outgoing_edges in edge_map.values():
        n = len(outgoing_edges)
        if n < 2:
            raise ValueError("Invalid DCEL. There should be at least two half edges for a vertex")
        
        for j in range(n):
            outgoing_edges[j].twin.next = outgoing_edges[(j + 1) % n]

    for he in tqdm(edges):
        if he.face is not None:
            continue
        
        face = Face()
        face.edge = he
        faces = np.append(faces, [face])
        
        current = he
        while True:
            current.face = face
            current = current.next
            if current is he:
                break

    return DCEL(vertex_objs, edges, faces)


In [13]:
def create_dcel_map(mesh_a, mesh_b):
    segment_map = defaultdict(set)

    vertices = np.concat((np.asarray(mesh_a.vertices), np.asarray(mesh_b.vertices)))
    segments_ = []

    for seg_a in tqdm(segments(mesh_a)):
        for seg_b in segments(mesh_b):
            seg_b = (seg_b[0] + len(mesh_a.vertices),
                     seg_b[1] + len(mesh_a.vertices))
            arc_inter = find_arcs_intersection(
                (vertices[seg_a[0]], vertices[seg_a[1]]),
                (vertices[seg_b[0]], vertices[seg_b[1]]),
            )

            segment_map[segment_hash(seg_a)].update((seg_a[0], seg_a[1]))
            segment_map[segment_hash(seg_b)].update((seg_b[0], seg_b[1]))

            if arc_inter is not None:
                vertices = np.append(vertices, [arc_inter], axis=0)
                segment_map[segment_hash(seg_a)].add(len(vertices) - 1)
                segment_map[segment_hash(seg_b)].add(len(vertices) - 1)

    for (start_idx, _), points in segment_map.items():
        points = sorted(points, key=lambda i: np.linalg.norm(
            np.cross(vertices[start_idx], vertices[i])))
        for i in range(len(points) - 1):
            segments_.append([points[i], points[i + 1]])

    # remove duplicate vertices
    vertices_map = {}
    new_vertices = []
    new_segments = set()

    for v in vertices:
        v_hash = vertex_hash(v)
        if v_hash not in vertices_map:
            vertices_map[v_hash] = len(new_vertices)
            new_vertices.append(v)

    for start, end in segments_:
        seg = (
            vertices_map[vertex_hash(vertices[start])],
            vertices_map[vertex_hash(vertices[end])]
        )

        if seg[0] == seg[1]:
            continue

        new_segments.add(segment_hash(seg))

    return build_dcel(new_vertices, new_segments)

In [14]:
def triangulate_face(face_vertices):
    # 1. Находим центр масс
    center = np.mean(face_vertices, axis=0)
    
    # 2. Проецируем на касательную плоскость
    tangent_plane = []
    for v in face_vertices:
        v_proj = v - center * np.dot(center, v)
        tangent_plane.append(v_proj[:2])  # Берём 2D координаты
    
    # 3. Триангуляция Делоне
    tri = Delaunay(tangent_plane)
    
    # 4. Возвращаем индексы треугольников
    return tri.simplices

In [15]:
def create_super_mesh(mesh_a, mesh_b):
    dcel_map = create_dcel_map(mesh_a, mesh_b)
    
    triangles = np.ndarray((0, 3), dtype=np.int32)
    for face in dcel_map.faces:
        vertex_indices = np.fromiter(face.vertex_indices(), dtype=int)
        face_vertices = dcel_map.vertices[vertex_indices]
        face_triangles = vertex_indices[triangulate_face(face_vertices)]
        triangles = np.append(triangles, face_triangles, axis=0)

    super_mesh = o3d.geometry.TriangleMesh()
    super_mesh.vertices = o3d.utility.Vector3dVector(dcel_map.vertices)
    super_mesh.triangles = o3d.utility.Vector3iVector(triangles)
    super_mesh.compute_vertex_normals()
    
    return super_mesh

In [16]:
parametrized_mesh_a = parametrize_mesh(mesh_a)
parametrized_mesh_b = parametrize_mesh(mesh_b)
overlap_meshes(parametrized_mesh_b, parametrized_mesh_a) #TODO: don't change mesh_b

super_mesh = create_super_mesh(parametrized_mesh_a, parametrized_mesh_b)

384it [01:15,  5.07it/s]
100%|██████████| 9632/9632 [00:00<00:00, 323851.76it/s]


In [27]:
def compute_barycentric(P, A, B, C):
    # Векторы от вершины C
    v0 = B - A
    v1 = C - A
    v2 = P - A
    
    # Площади через векторное произведение
    denom = v0[0] * v1[1] - v1[0] * v0[1]
    alpha = (v2[0] * v1[1] - v1[0] * v2[1]) / denom
    beta = (v0[0] * v2[1] - v2[0] * v0[1]) / denom
    gamma = 1 - alpha - beta
    return np.array([alpha, beta, gamma])

def find_enclosing_triangle(point, mesh):
    vertices = np.asarray(mesh.vertices)
    for tri in np.asarray(mesh.triangles):
        tri_vertices = vertices[tri]
        bary = compute_barycentric(point, *tri_vertices)
        if np.all(np.greater_equal(bary, 0)):
            return tri, bary
    raise RuntimeError("Could not find any correspoinding triangle which is impossible")

# def build_vertices_paths(mesh_a, parametrized_mesh_a, mesh_b, parametrized_mesh_b, super_mesh):


In [29]:
source_vertices = np.asarray(mesh_a.vertices)
target_vertices = np.asarray(mesh_b.vertices)

v_source = np.zeros((len(super_mesh.vertices), 3))
v_target = np.zeros((len(super_mesh.vertices), 3))

for i, v in tqdm(enumerate(np.asarray(super_mesh.vertices))):
    src_tri, src_bary = find_enclosing_triangle(v, parametrized_mesh_a)
    v_source[i] = np.sum(source_vertices[src_tri] * src_bary, axis=0)

    tgt_tri, tgt_bary = find_enclosing_triangle(v, parametrized_mesh_b)
    v_target[i] = np.sum(target_vertices[tgt_tri] * tgt_bary, axis=0)
    

17it [00:00, 82.88it/s]


RuntimeError: Could not find any correspoinding triangle which is impossible

In [ ]:
geo = []

add_wireframe_to_geo(parametrized_mesh_a, geo, [0, 0, 1])
add_wireframe_to_geo(parametrized_mesh_b, geo, [1, 0, 0])
add_wireframe_to_geo(super_mesh, geo)

o3d.visualization.draw_geometries(geo, mesh_show_wireframe=True)

2025-05-31 13:24:20.311 python[27959:3063471] +[IMKClient subclass]: chose IMKClient_Modern
2025-05-31 13:24:20.312 python[27959:3063471] +[IMKInputSession subclass]: chose IMKInputSession_Modern
